In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv

In [ ]:
def fast(file, nms_file, non_nms_file):
    img = cv.imread(file,0)

    # Initiate FAST object with default values
    fast = cv.FastFeatureDetector_create()

    # find and draw the keypoints
    kp = fast.detect(img,None)
    img2 = cv.drawKeypoints(img, kp, None, color=(255,0,0))

    # Print all default params
    print( "Threshold: {}".format(fast.getThreshold()) )
    print( "nonmaxSuppression:{}".format(fast.getNonmaxSuppression()) )
    print( "neighborhood: {}".format(fast.getType()) )
    print( "Total Keypoints with nonmaxSuppression: {}".format(len(kp)) )
    cv.imwrite(nms_file,img2)

    # Disable nonmaxSuppression
    fast.setNonmaxSuppression(0)
    kp = fast.detect(img,None)
    print( "Total Keypoints without nonmaxSuppression: {}".format(len(kp)) )
    img3 = cv.drawKeypoints(img, kp, None, color=(255,0,0))
    cv.imwrite(non_nms_file,img3)

def display(img1, img2, title1, title2):
    fig, axs = plt.subplots(1, 2, figsize = (15, 7))
    axs[0].imshow(img1)
    axs[1].imshow(img2)
    axs[0].set_title(title1)
    axs[1].set_title(title2)
    
def display_files(file1, file2, title1='', title2=''):
    img1 = mpimg.imread(file1)
    img2 = mpimg.imread(file2)
    fig, axs = plt.subplots(1, 2, figsize = (15, 7))
    axs[0].imshow(img1, cmap='gray')
    axs[1].imshow(img2, cmap='gray')
    axs[0].set_title(title1)
    axs[1].set_title(title2)
    
def orb(file):
    img = cv.imread(file,0)
    
    # Initiate ORB detector
    orb = cv.ORB_create()
    
    # find the keypoints with ORB
    kp = orb.detect(img,None)
    
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    
    # draw only keypoints location,not size and orientation
    img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
    plt.figure(figsize=(15,15)), plt.imshow(img2), plt.show()
    
def feature_matching_bf_orb(file1, file2):
    img1 = cv.imread(file1,cv.IMREAD_GRAYSCALE)          # queryImage
    img2 = cv.imread(file2,cv.IMREAD_GRAYSCALE) # trainImage
    
    # Initiate ORB detector
    orb = cv.ORB_create()
    
    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    
    # create BFMatcher object
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    
    # Match descriptors.
    matches = bf.match(des1,des2)
    
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    # Draw first 10 matches.
    img3 = cv.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(20,10)), plt.imshow(img3),plt.show()
    
def feature_matching_bf_sift(file1, file2):
    img1 = cv.imread(file1,cv.IMREAD_GRAYSCALE)          # queryImage
    img2 = cv.imread(file2,cv.IMREAD_GRAYSCALE) # trainImage
    
    # Initiate SIFT detector
    sift = cv.xfeatures2d.SIFT_create()
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    
    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
            
    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good[:10],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(20,10)), plt.imshow(img3),plt.show()
    
def feature_matching_flann_sift(file1, file2):
    img1 = cv.imread(file1,cv.IMREAD_GRAYSCALE)          # queryImage
    img2 = cv.imread(file2,cv.IMREAD_GRAYSCALE) # trainImage
    
    # Initiate SIFT detector
    sift = cv.xfeatures2d.SIFT_create()
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    
#     # Need to draw only good matches, so create a mask
#     matchesMask = [[0,0] for i in range(len(matches))]
    
#     # ratio test as per Lowe's paper
#     for i,(m,n) in enumerate(matches):
#         if m.distance < 0.7*n.distance:
#             matchesMask[i]=[1,0]
            
#     draw_params = dict(matchColor = (0,255,0),
#                        singlePointColor = (255,0,0),
#                        matchesMask = matchesMask[:10],
#                        flags = cv.DrawMatchesFlags_DEFAULT)
    
#     img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches[:10],None,**draw_params)
#     plt.figure(figsize=(20,10)), plt.imshow(img3),plt.show()
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
            
    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good[:30],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(20,10)), plt.imshow(img3),plt.show()
    
    
def down_sp(file, down1_file, down2_file, down3_file):
    src = cv.imread(file, cv.IMREAD_GRAYSCALE)
    print('Original shape: ', src.shape)
    
    rows, cols = map(int, src.shape)
    down1 = cv.pyrDown(src, dstsize=(cols // 2, rows // 2))
    print('Down1 shape: ', down1.shape)
    cv.imwrite(down1_file, down1)
    
    rows, cols = map(int, down1.shape)
    down2 = cv.pyrDown(down1, dstsize=(cols // 2, rows // 2))
    print('Down2 shape: ', down2.shape)
    cv.imwrite(down2_file, down2)
    
    rows, cols = map(int, down2.shape)
    down3 = cv.pyrDown(down2, dstsize=(cols // 2, rows // 2))
    print('Down2 shape: ', down3.shape)
    cv.imwrite(down3_file, down3)
    
    plt.figure(figsize=(10,10)), plt.imshow(down3), plt.show()
    
def detect_moving_obj(file1, file2):
    """
    Extract the features of self.prev_gray and cur_gray using FAST detector and BRIEF
    :param cur_gray: the input grayscale image
    :return: key points and descriptors of self.prev_gray and cur_gray
    """
    fast = cv.xfeatures2d.FAST_create()

    # find the keypoints with FAST and descriptors of self.prev_gray
    kp1 = fast.detect(self.prev_gray, None)
    kp1, des1 = brief.compute(self.prev_gray, kp1[:200])

    # find the keypoints with FAST and descriptors of cur_gray
    kp2 = self.fast.detect(cur_gray, None)
    kp2, des2 = self.brief.compute(cur_gray, kp2[:200])

    return kp1, des1, kp2, des2

In [ ]:
fast('images/1.jpg', 'images/fast/1_true.png', 'images/fast/1_false.png')

In [ ]:
display_files('images/1_true.png', 'images/1_false.png', '1_true', '1_false')

In [ ]:
orb('images/1.jpg')

In [ ]:
feature_matching_bf_orb('images/box1.jpg', 'images/box2.jpg')

In [ ]:
down_sp('images/1.jpg', 'images/1_down1.jpg', 'images/1_down2.jpg', 'images/1_down3.jpg')

In [ ]:
down_sp('images/2.jpg', 'images/2_down1.jpg', 'images/2_down2.jpg', 'images/2_down3.jpg')

In [ ]:
down_sp('images/box1.jpg', 'images/box1_down1.jpg', 'images/box1_down2.jpg', 'images/box1_down3.jpg')

In [ ]:
down_sp('images/box2.jpg', 'images/box2_down1.jpg', 'images/box2_down2.jpg', 'images/box2_down3.jpg')

In [ ]:
feature_matching_bf_orb('images/box1_down3.jpg', 'images/box2_down3.jpg')

In [ ]:
feature_matching_bf_orb('images/1_down3.jpg', 'images/2_down3.jpg')

In [ ]:
feature_matching_bf_sift('images/box1_down3.jpg', 'images/box2_down3.jpg')

In [ ]:
feature_matching_bf_sift('images/1_down3.jpg', 'images/2_down3.jpg')

In [ ]:
feature_matching_flann_sift('images/1_down3.jpg', 'images/2_down3.jpg')

In [ ]:
feature_matching_flann_sift('images/box1_down3.jpg', 'images/box2_down3.jpg')

In [ ]:
down_sp('images/ball1.jpg', 'images/ball1_down1.jpg', 'images/ball1_down2.jpg', 'images/ball1_down3.jpg')

In [ ]:
down_sp('images/ball2.jpg', 'images/ball2_down1.jpg', 'images/ball2_down2.jpg', 'images/ball2_down3.jpg')

In [ ]:
down_sp('images/book1.jpg', 'images/book1_down1.jpg', 'images/book1_down2.jpg', 'images/book1_down3.jpg')

In [ ]:
down_sp('images/book2.jpg', 'images/book2_down1.jpg', 'images/book2_down2.jpg', 'images/book2_down3.jpg')

In [ ]:
down_sp('images/moving_bottle1.jpg', 'images/moving_bottle1_down1.jpg', 'images/moving_bottle1_down2.jpg', 'images/moving_bottle1_down3.jpg')

In [ ]:
down_sp('images/moving_bottle2.jpg', 'images/moving_bottle2_down1.jpg', 'images/moving_bottle2_down2.jpg', 'images/moving_bottle2_down3.jpg')